In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as f

spark = SparkSession.builder.getOrCreate()

22/08/16 12:27:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/16 12:27:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:

# 4
olist_order_payments_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_order_payments_dataset.csv")
# 6
olist_orders_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_orders_dataset.csv")
# 1
olist_customers_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',')\
                               .csv('data/olist_customers_dataset.csv')

In [10]:
olist_customers_dataset.show()


+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [21]:
print(f"Distinct values in column customer_id: {olist_customers_dataset.select('customer_id').distinct().count()}")
print(f"Null values in column customer_id: {olist_customers_dataset.where(f.col('customer_id').isNull()).count()}")


Distinct values in column customer_id: 99441
Null values in column customer_id: 0


In [22]:
print(f"Distinct values in column customer_id: {olist_customers_dataset.select('customer_unique_id').distinct().count()}")
print(f"Null values in column customer_id: {olist_customers_dataset.where(f.col('customer_unique_id').isNull()).count()}")

Distinct values in column customer_id: 96096
Null values in column customer_id: 0


+-------------------------------------------------+
|count(CASE WHEN isnan(customer_id) THEN true END)|
+-------------------------------------------------+
|                                                0|
+-------------------------------------------------+



In [28]:
olist_customers_dataset.groupBy("customer_unique_id","customer_zip_code_prefix", "customer_city", "customer_state").count().filter("count > 1").show(truncate=False)

+--------------------------------+------------------------+-------------------+--------------+-----+
|customer_unique_id              |customer_zip_code_prefix|customer_city      |customer_state|count|
+--------------------------------+------------------------+-------------------+--------------+-----+
|44bd7571a7dad968306f7776bd5a5745|7500                    |santa isabel       |SP            |2    |
|ceb767afcf00ef4fedae42343064aa3d|5171                    |sao paulo          |SP            |2    |
|76c9a12722537319e44c31e70b7815c3|13960                   |socorro            |SP            |2    |
|e5b6fb6c5e7e4adc25a5e400c0104508|88070                   |florianopolis      |SC            |2    |
|93eeb8745a219d35b7e32d796528b681|24030                   |niteroi            |RJ            |2    |
|af842a6ed9f77511c9bf46922a29ecb5|13140                   |paulinia           |SP            |2    |
|addc1082e059523e5ef2e7486d1d8905|3456                    |sao paulo          |SP          

In [31]:
olist_customers_dataset.filter("customer_unique_id == 'ab744045d2445f5bae53b57a4b99bfbf'").show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|0598384cf66a6bd49ac0c725af1c74e7|ab744045d2445f5bae53b57a4b99bfbf|9240                    |santo andre  |SP            |
|6ca87302fa8a0163488bce8e49f9996c|ab744045d2445f5bae53b57a4b99bfbf|9240                    |santo andre  |SP            |
|9e80bf15286ac4efa8833654940b0e1f|ab744045d2445f5bae53b57a4b99bfbf|9240                    |santo andre  |SP            |
+--------------------------------+--------------------------------+------------------------+-------------+--------------+



In [32]:
olist_customers_dataset.filter("customer_unique_id == 'd7a86208b8dd2e697ddf825c386a0402'").show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city      |customer_state|
+--------------------------------+--------------------------------+------------------------+-------------------+--------------+
|e2d7302d9765c89609899067c37f4722|d7a86208b8dd2e697ddf825c386a0402|12246                   |sao jose dos campos|SP            |
|f976a6aa7c97f6713c9b39e5bd74e04d|d7a86208b8dd2e697ddf825c386a0402|12246                   |sao jose dos campos|SP            |
+--------------------------------+--------------------------------+------------------------+-------------------+--------------+



In [33]:
olist_customers_dataset.filter("customer_unique_id == '93eeb8745a219d35b7e32d796528b681'").show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|210b65c1eb5a1df0da71126a258cfdc3|93eeb8745a219d35b7e32d796528b681|24030                   |niteroi      |RJ            |
|7f1a08e2635252e5e256726471ef63ad|93eeb8745a219d35b7e32d796528b681|24030                   |niteroi      |RJ            |
+--------------------------------+--------------------------------+------------------------+-------------+--------------+



In [34]:
olist_customers_dataset.filter("customer_unique_id == 'af842a6ed9f77511c9bf46922a29ecb5'").show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|3cd215d1f5e1965177ecbce1f46dbdad|af842a6ed9f77511c9bf46922a29ecb5|13140                   |paulinia     |SP            |
|1c901dd8f6bccda64fe8582fe3dd18a8|af842a6ed9f77511c9bf46922a29ecb5|13140                   |paulinia     |SP            |
+--------------------------------+--------------------------------+------------------------+-------------+--------------+



In [35]:
olist_geolocation_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_geolocation_dataset.csv")


In [36]:
olist_geolocation_dataset.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1037| -23.54562128115268| -46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535| -46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469| -46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681| -46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493| -46.64160722329613|       sao paulo|               SP|
|                       1012|-23.547762303364266| -46.63536053788448|       são paulo|               SP|
|                       1047|-23.546273112412678| -46.6

In [38]:
olist_geolocation_dataset.groupBy("geolocation_zip_code_prefix","geolocation_lat","geolocation_lng", "geolocation_city", "geolocation_state").count().filter("count > 1").show(truncate=False)

+---------------------------+-------------------+-------------------+----------------+-----------------+-----+
|geolocation_zip_code_prefix|geolocation_lat    |geolocation_lng    |geolocation_city|geolocation_state|count|
+---------------------------+-------------------+-------------------+----------------+-----------------+-----+
|1026                       |-23.53925652899767 |-46.633440525535235|sao paulo       |SP               |2    |
|1106                       |-23.531062064948905|-46.629505278925556|sao paulo       |SP               |2    |
|1241                       |-23.545792634196992|-46.6567234730798  |sao paulo       |SP               |3    |
|1226                       |-23.538190850683794|-46.651323227306854|são paulo       |SP               |4    |
|1223                       |-23.543735510964265|-46.65415427699233 |sao paulo       |SP               |2    |
|1303                       |-23.551504721622702|-46.64945164585375 |sao paulo       |SP               |3    |
|

In [42]:
olist_geolocation_dataset.filter("geolocation_zip_code_prefix == '24030'").show(truncate=False)

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat    |geolocation_lng    |geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|24030                      |-22.883862923138043|-43.11562368659535 |niterói         |RJ               |
|24030                      |-22.882993846173793|-43.12165872133218 |niteroi         |RJ               |
|24030                      |-22.890436234928647|-43.12244643342076 |niteroi         |RJ               |
|24030                      |-22.887588762132903|-43.11781399056493 |niteroi         |RJ               |
|24030                      |-22.892535283368957|-43.113701585175356|niteroi         |RJ               |
|24030                      |-22.884583788747086|-43.121444329332505|niterói         |RJ               |
|24030                      |-22.885338477347375|-43.11

In [41]:
olist_customers_dataset.filter("customer_zip_code_prefix == '24030'").show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|da89768d5e47c76c1b3f2694e6c8f6a9|3bfe238d63378a92a404dc5c8ca31944|24030                   |niteroi      |RJ            |
|71fdbaa760e25ef2449d79d1c4264021|776469358dc5cbca9e8987fa110d804a|24030                   |niteroi      |RJ            |
|315d30072434c1feda91ba8676b54a25|22eecffbc2e4e56747eb329d96051962|24030                   |niteroi      |RJ            |
|c1d3f90ae6566b0d6e0673d6a2de44d0|6856178ce177496369e3a83fd143e621|24030                   |niteroi      |RJ            |
|8e6e118324e9b1a48fb079c5eee2d6ff|35c75f00917e33d1db28551ee906324e|24030                   |niteroi      |RJ            |
|7f16d43cea70ede77ed4268